In [1]:
import random
from collections import Counter

# ----------------------------
# Domain knowledge (ecology)
# ----------------------------

PLANT_SPECIES = {
    "grass": {
        "edibility": 0.8,
        "growth_rate": 0.20
    },
    "clover": {
        "edibility": 0.6,
        "growth_rate": 0.15
    },
    "shrub": {
        "edibility": 0.2,
        "growth_rate": 0.05
    }
}

ANIMAL_SPECIES = {
    "rabbit": ["grass", "clover"],
    "deer": ["grass", "clover", "shrub"]
}

LOCATIONS = ["meadow", "forest_edge", "clearing"]

TIME_STEPS = 6
NUM_TRACES = 6000


# ----------------------------
# Object definitions
# ----------------------------

class Plant:
    def __init__(self, pid):
        self.id = pid
        self.species = random.choice(list(PLANT_SPECIES.keys()))
        self.location = random.choice(LOCATIONS)
        self.alive = True
        self.eaten = False

    def step(self):
        """Plants may grow (stay alive) or die naturally"""
        if not self.alive:
            return

        # Natural death
        if random.random() < 0.03:
            self.alive = False

        # Regrowth / resilience modeled implicitly
        # (survival probability encoded in low death rate)


class Animal:
    def __init__(self, aid):
        self.id = aid
        self.species = random.choice(list(ANIMAL_SPECIES.keys()))
        self.location = random.choice(LOCATIONS)

    def move(self):
        self.location = random.choice(LOCATIONS)

    def try_eat(self, plants):
        """Attempt to eat a compatible plant in the same location"""
        for plant in plants:
            if (
                plant.alive
                and plant.location == self.location
                and plant.species in ANIMAL_SPECIES[self.species]
            ):
                p_eat = PLANT_SPECIES[plant.species]["edibility"]
                if random.random() < p_eat:
                    plant.alive = False
                    plant.eaten = True
                    return True
        return False


# ----------------------------
# World simulation
# ----------------------------

def simulate_world():
    num_animals = random.randint(1, 5)
    num_plants = random.randint(2, 6)

    animals = [Animal(i) for i in range(num_animals)]
    plants = [Plant(i) for i in range(num_plants)]

    eaten_any = False

    for _ in range(TIME_STEPS):
        for animal in animals:
            animal.move()
            if animal.try_eat(plants):
                eaten_any = True

        for plant in plants:
            plant.step()

    return {
        "plants": plants,
        "eaten_any": eaten_any
    }


# ----------------------------
# Conditioning & inference
# ----------------------------

def run_inference():
    conditioned = []

    for _ in range(NUM_TRACES):
        trace = simulate_world()
        if trace["eaten_any"]:  # observation
            conditioned.append(trace)

    return conditioned


def estimate_probabilities(traces):
    plant0_eaten = 0
    majority_survive = 0

    for trace in traces:
        plants = trace["plants"]
        alive = sum(p.alive for p in plants)

        for p in plants:
            if p.id == 0 and p.eaten:
                plant0_eaten += 1

        if alive > len(plants) / 2:
            majority_survive += 1

    total = len(traces)

    return {
        "P(plant 0 eaten | eating observed)": plant0_eaten / total,
        "P(>50% plants survive | eating observed)": majority_survive / total,
        "Conditioned traces": total
    }


 #----------------------------
# Main
# ----------------------------

if __name__ == "__main__":
    traces = run_inference()
    results = estimate_probabilities(traces)

    print("Conditioned on: at least one animal ate a plant\n")
    for k, v in results.items():
        print(f"{k}: {v:.3f}" if isinstance(v, float) else f"{k}: {v}")


Conditioned on: at least one animal ate a plant

P(plant 0 eaten | eating observed): 0.770
P(>50% plants survive | eating observed): 0.059
Conditioned traces: 5846


# Probabilistic World Simulation: Ecosystem Model

## How Random Variables and Relations are Represented

- **Entity existence:** Number of animals (1–5) and plants (2–6) are randomly sampled at the start of each simulation.  
- **Entity attributes:** Plant species, animal species, and initial locations are randomly assigned per entity.  
- **World dynamics:** Animals move stochastically, plants may die naturally, and animals attempt to eat plants compatible with their diet.  
- **Relational structure:** Eating events depend on the interaction between animals and plants in the same location and diet compatibility.

---

## How Unknown Numbers of Objects are Handled

- The simulation uses **open-universe modeling**:  
  - Each run can have a different number of animals and plants.  
  - Each run samples species and location assignments independently.  
  - This creates a “possible world” for each trace.

---

## How Observations Affect Probabilities

- We can **condition** the simulation on the observation that **at least one animal ate a plant**.  
- Execution traces that do not satisfy this observation are discarded.  
- As a result:  
  - The probability of a specific plant being eaten increases.  
  - The probability of most plants surviving decreases.  

- This demonstrates **Bayesian reasoning under uncertainty**, using execution traces to estimate posterior probabilities.




 # Probabilistic Modeling of Student Movement on Campus

 1. Scope for a Semester-Long Project

This project aims to model and predict student movement across campus **after their scheduled activities**. Students are categorized as athletes, dancers, or regular students. The project will use **probabilistic and relational modeling** to simulate how students move between campus locations (e.g., cafeteria, library, lounges, outdoor areas) over time.  

Key objectives:
- Represent student populations with uncertain behavior.
- Capture relational dependencies (friends, teammates influence movement).
- Simulate dynamic evolution of locations over time steps.
- Condition predictions on partial observations (e.g., some students already observed at a location).

This simulation will generate **thousands of traces** to estimate probabilities and identify patterns of student movement.

---

 2. Problem Statement

Even with knowledge of student schedules, the exact **post-activity movement** is uncertain. Students make decisions influenced by personal preference, social groups, and environmental context.  

The problem is to **predict probabilistically where students are likely to go**, accounting for:
- Time-of-day effects
- Social influence from friends or teammates
- Stochastic movement choices

This allows the simulation to answer questions like:
- What is the probability that most athletes will go to the cafeteria at 1 PM?
- How does observing a few students at a location update predictions for the rest?

---

3. Relevance of AI Methodologies from the Chapter

From *Artificial Intelligence: A Modern Approach*, the following concepts apply:

- **Open-Universe Modeling**: The number and behavior of students in each location is uncertain; Monte Carlo traces sample possible worlds.
- **Dynamic Models**: Student locations evolve over discrete time steps, forming a stochastic transition process.
- **Relational Probabilistic Models**: Friend and team influences create dependencies in movement probabilities.
- **Partial Observations / Conditioning**: Observing some students updates posterior probabilities for others.
- **Monte Carlo Simulation**: Used to approximate distributions over student locations across multiple traces.
- **Bayesian Reasoning**: Inference over likely student positions given partial observations.

These methodologies allow us to **reason under uncertainty** and simulate complex, relationally-dependent behavior on campus.

---

 4. Example Python Code

The code below demonstrates **dynamic probabilistic simulation**, relational dependencies, and conditioning on observations using Monte Carlo simulation.


In [ ]:
import random

# ----------------------------
# Campus & Student Domain
# ----------------------------

STUDENT_TYPES = ["athlete", "dancer", "regular"]
LOCATIONS = ["cafeteria", "library", "lounge", "gym", "outdoors"]

# Example type preferences per time step (simplified)
STUDENT_TYPE_PREFERENCES = {
    "athlete": [["gym","cafeteria"], ["cafeteria","lounge"], LOCATIONS],
    "dancer": [["studio","cafeteria"], ["cafeteria","lounge"], LOCATIONS],
    "regular": [LOCATIONS, LOCATIONS, LOCATIONS]
}

class Student:
    def __init__(self, sid, stype, friends=[]):
        self.id = sid
        self.type = stype
        self.friends = friends
        self.location = None
        
    def move(self, student_locations, time_step):
        # Base probabilities by type
        possible = STUDENT_TYPE_PREFERENCES[self.type][time_step]
        # Add social influence from friends
        friend_locs = [loc for s, loc in student_locations if s in self.friends]
        if friend_locs:
            possible += friend_locs  # more likely to follow friends
        self.location = random.choice(possible)

# ----------------------------
# Simulation Functions
# ----------------------------

def simulate_day(students, time_steps=3):
    traces = []
    for t in range(time_steps):
        step_locations = []
        for s in students:
            s.move(step_locations, t)
            step_locations.append((s.id, s.location))
        traces.append(step_locations)
    return traces

def monte_carlo_simulation(students, num_traces=1000, observed=None):
    all_traces = []
    for _ in range(num_traces):
        traces = simulate_day(students)
        if observed:
            # Keep only traces consistent with observations
            consistent = True
            for obs in observed:
                sid, time, loc = obs['student'], obs['time'], obs['location']
                if traces[time][sid][1] != loc:
                    consistent = False
                    break
            if consistent:
                all_traces.append(traces)
        else:
            all_traces.append(traces)
    return all_traces

def estimate_probability(traces, student_type, location, time_step):
    count = 0
    for trace in traces:
        students_in_location = sum(1 for sid, loc in trace[time_step] 
                                   if sid.startswith(student_type[0]) and loc == location)
        if students_in_location > len(trace[time_step])/2:
            count += 1
    return count / len(traces)

# ----------------------------
# Example Execution
# ----------------------------

# Create students
students = [Student(f"a{i}", "athlete", friends=[f"a{(i+1)%3}"]) for i in range(3)]
students += [Student(f"d{i}", "dancer", friends=[f"d{(i+1)%2}"]) for i in range(2)]
students += [Student(f"r{i}", "regular") for i in range(5)]

# Run Monte Carlo simulation
traces = monte_carlo_simulation(students, num_traces=200)

# Estimate probability most athletes are in cafeteria at time step 1
prob = estimate_probability(traces, "athlete", "cafeteria", 1)
print(f"Probability that most athletes are in cafeteria at time step 1: {prob:.3f}")
